In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'tube_size_SAE'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='TubeSize'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

2275


In [5]:
# df[field].explode().value_counts()
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, TubeSize]
Index: []

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [7]:
df[field].explode().value_counts()

10mm    70
12mm    44
8mm     34
20mm    26
22mm    23
15mm    23
18mm    21
16mm    18
14mm    16
25mm    11
28mm    11
30mm     9
6mm      5
Name: TubeSize, dtype: int64

In [17]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(tube)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(?i)(\d*.?\d+\/\d+".{0,3}\d*.?\d*\/?\d*"?)|(\d+")|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

pat=r'(?i)(m)|()'
df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm',' mm',str(x))).apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r'\.0','',str(x))).apply(lambda x: re.sub(r'(?i)\s?Am?p?s?','A',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col].apply(lambda x: re.sub(r'"?\s?x\s?','" - ',str(x)))#.apply(lambda x: f'{x}')
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
matchcustom[curation_col].explode().value_counts()
custom

311


buckets external_id  \
363   Compressors Accessories & Replacement Parts      08U45T   
369   Compressors Accessories & Replacement Parts      10U50T   
562   Compressors Accessories & Replacement Parts      12U52T   
572   Compressors Accessories & Replacement Parts    GT370406   
573   Compressors Accessories & Replacement Parts    GT370408   
574   Compressors Accessories & Replacement Parts    GT370410   
575   Compressors Accessories & Replacement Parts    GT370610   
576   Compressors Accessories & Replacement Parts    GT370612   
577   Compressors Accessories & Replacement Parts    GT370815   
578   Compressors Accessories & Replacement Parts    GT371018   
579   Compressors Accessories & Replacement Parts    GT371222   
580   Compressors Accessories & Replacement Parts    GT371628   
581   Compressors Accessories & Replacement Parts    GT380408   
582   Compressors Accessories & Replacement Parts    GT380410   
583   Compressors Accessories & Replacement Parts    GT380610   
584   Compressors Accessories & Replacement Parts    GT380612   
585   Compressors Accessories & Replacement Parts    GT380614   
586   Compressors Accessories & Replacement Parts    GT380816   
587   Compressors Accessories & Replacement Parts    GT381020   
588   Compressors Accessories & Replacement Parts    GT381220   
589   Compressors Accessories & Replacement Parts    GT381225   
590   Compressors Accessories & Replacement Parts    GT381630   
597   Compressors Accessories & Replacement Parts    GT410408   
598   Compressors Accessories & Replacement Parts    GT410410   
599   Compressors Accessories & Replacement Parts    GT410610   
600   Compressors Accessories & Replacement Parts    GT410612   
601   Compressors Accessories & Replacement Parts    GT410815   
602   Compressors Accessories & Replacement Parts    GT411018   
603   Compressors Accessories & Replacement Parts    GT411222   
604   Compressors Accessories & Replacement Parts    GT420410   
605   Compressors Accessories & Replacement Parts    GT420612   
606   Compressors Accessories & Replacement Parts    GT420816   
607   Compressors Accessories & Replacement Parts    GT421020   
612   Compressors Accessories & Replacement Parts    GT440408   
613   Compressors Accessories & Replacement Parts    GT440410   
614   Compressors Accessories & Replacement Parts    GT440610   
615   Compressors Accessories & Replacement Parts    GT440612   
616   Compressors Accessories & Replacement Parts    GT440815   
617   Compressors Accessories & Replacement Parts    GT441018   
618   Compressors Accessories & Replacement Parts    GT441222   
619   Compressors Accessories & Replacement Parts    GT450410   
620   Compressors Accessories & Replacement Parts    GT450612   
621   Compressors Accessories & Replacement Parts    GT450614   
622   Compressors Accessories & Replacement Parts    GT450816   
623   Compressors Accessories & Replacement Parts    GT451020   
624   Compressors Accessories & Replacement Parts      12U55T   
648   Compressors Accessories & Replacement Parts    GT660408   
649   Compressors Accessories & Replacement Parts    GT660610   
650   Compressors Accessories & Replacement Parts    GT660612   
651   Compressors Accessories & Replacement Parts    GT660815   
652   Compressors Accessories & Replacement Parts    GT661018   
653   Compressors Accessories & Replacement Parts    GT661222   
654   Compressors Accessories & Replacement Parts    GT670408   
655   Compressors Accessories & Replacement Parts    GT670410   
656   Compressors Accessories & Replacement Parts    GT670610   
657   Compressors Accessories & Replacement Parts    GT670612   
658   Compressors Accessories & Replacement Parts    GT670614   
659   Compressors Accessories & Replacement Parts    GT670816   
660   Compressors Accessories & Replacement Parts    GT671020   
661   Compressors Accessories & Replacement Parts      04U38T   
663   Compressors Accessories & Replacement Parts      06U40T   
664   Compre

In [9]:
not_match=df[(df[field].astype(str)=='nan')]
# print(len(not_match[(not_match['ld'].astype(str)=='[]')|(not_match['name'].astype(str)!='[]')]))
two=not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]
# print(len(two))
# two[curation_col]=two['name'].apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))


two[curation_col]=two['ld'].apply(lambda x: re.sub(r'(?i)\s?amp','A',str(x)))
matchtwo=two[['external_id',curation_col]]
print(len(matchtwo))
two

61


buckets external_id  \
321   Compressors Accessories & Replacement Parts   10408N668   
372   Compressors Accessories & Replacement Parts   4S12FHA12   
373   Compressors Accessories & Replacement Parts   4S12FHB12   
387   Compressors Accessories & Replacement Parts   4S16FHA16   
388   Compressors Accessories & Replacement Parts   4S16FHB12   
389   Compressors Accessories & Replacement Parts   4S16FHB16   
410   Compressors Accessories & Replacement Parts   4S20FHA20   
411   Compressors Accessories & Replacement Parts   4S20FHB16   
412   Compressors Accessories & Replacement Parts   4S20FHB20   
428   Compressors Accessories & Replacement Parts   4S24FHB20   
429   Compressors Accessories & Replacement Parts   4S24FHB24   
1061  Compressors Accessories & Replacement Parts  43016U8046   
1414  Compressors Accessories & Replacement Parts   K43016D76   
1415  Compressors Accessories & Replacement Parts   K43016G76   
1418  Compressors Accessories & Replacement Parts   K43016G46   
1419  Compressors Accessories & Replacement Parts  K43016G764   
1421  Compressors Accessories & Replacement Parts   K43020G50   
1422  Compressors Accessories & Replacement Parts   K43012G72   
1423  Compressors Accessories & Replacement Parts   K43012G42   
1425  Compressors Accessories & Replacement Parts   K43016G80   
1428  Compressors Accessories & Replacement Parts   K43016D80   
1429  Compressors Accessories & Replacement Parts   K43020G80   
1430  Compressors Accessories & Replacement Parts   K43012D72   
1431  Compressors Accessories & Replacement Parts  K43012G723   
1434  Compressors Accessories & Replacement Parts   K43020H54   
1437  Compressors Accessories & Replacement Parts   K43020D80   
1439  Compressors Accessories & Replacement Parts   K43012G76   
1440  Compressors Accessories & Replacement Parts   K43024G84   
1444  Compressors Accessories & Replacement Parts   K43016D46   
1445  Compressors Accessories & Replacement Parts   K43020H04   
1446  Compressors Accessories & Replacement Parts   K43020H24   
1447  Compressors Accessories & Replacement Parts   K43016G50   
1450  Compressors Accessories & Replacement Parts   K43008G71   
1451  Compressors Accessories & Replacement Parts  K430208040   
1453  Compressors Accessories & Replacement Parts   K43032G92   
1454  Compressors Accessories & Replacement Parts   K43020D50   
1455  Compressors Accessories & Replacement Parts   K43012D42   
1456  Compressors Accessories & Replacement Parts   K43012G46   
1457  Compressors Accessories & Replacement Parts   K43012H62   
1458  Compressors Accessories & Replacement Parts   K43012H52   
1459  Compressors Accessories & Replacement Parts   K43016H03   
1461  Compressors Accessories & Replacement Parts   K43008G41   
1462  Compressors Accessories & Replacement Parts   K43012H02   
1464  Compressors Accessories & Replacement Parts   K43016H53   
1469  Compressors Accessories & Replacement Parts   K43016H63   
1470  Compressors Accessories & Replacement Parts   K43016H23   
1471  Compressors Accessories & Replacement Parts   K43012H22   
1472  Compressors Accessories & Replacement Parts   K43012D76   
1473  Compressors Accessories & Replacement Parts   K43008G72   
1474  Compressors Accessories & Replacement Parts   K43008G42   
1476  Compressors Accessories & Replacement Parts   K43012H03   
1477  Compressors Accessories & Replacement Parts   K43012H53   
1478  Compressors Accessories & Replacement Parts   K43012H63   
1479  Compressors Accessories & Replacement Parts   K43032G62   
1480  Compressors Accessories & Replacement Parts   K43016H64   
1742  Compressors Accessories & Replacement Parts       50936   
1744  Compressors Accessories & Replacement Parts       50934   
1745  Compressors Accessories & Replacement Parts       50932   
1747  Compressors Accessories & Replacement Parts       50914   
1750  Compressors Accessories & Replacement Parts       50916   
1752  Compressors Accessories & Replacement Parts       50912   

           

In [10]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [11]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [12]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [18]:
na=df[(df['name'].astype(str)=='[]')&(df[field].astype(str)=='nan')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))
na

1903


buckets external_id  \
0     Compressors Accessories & Replacement Parts       40511   
1     Compressors Accessories & Replacement Parts       40490   
2     Compressors Accessories & Replacement Parts       40492   
3     Compressors Accessories & Replacement Parts       40491   
4     Compressors Accessories & Replacement Parts       40493   
...                                           ...         ...   
2270  Compressors Accessories & Replacement Parts    405784RE   
2271  Compressors Accessories & Replacement Parts    405786RE   
2272  Compressors Accessories & Replacement Parts    405787RE   
2273  Compressors Accessories & Replacement Parts     40578RE   
2274  Compressors Accessories & Replacement Parts     509005U   

                                                                         product_name  \
0                                                        1/16 Viton O-Ring Assortment   
1                                              1/16 - 1/8 Buna-N 90 O-Ring Assortment   
2                                                    3/32 Buna-N 90 O-Ring Assortment   
3                                                    1/16 Buna-N 90 O-Ring Assortment   
4                                                     1/8 Buna-N 90 O-Ring Assortment   
...                                                                               ...   
2270       Refill Kit for 1/4 - 5/8 Kim-Krimp 'Z' Series Face Seal Fitting Assortment   
2271                 Refill Kit for 1/4 - 1/2 Kim-Krimp 'Z' Series Fitting Assortment   
2272          Refill Kit for 1/4 - 1/2 Kim-Krimp 'Z' Series O-Ring Fitting Assortment   
2273  Refill Kit for Straight Thread O-Ring Kim-Krimp Hose End Assortment (1/4 - 1/2)   
2274                             Buna-N 90 O-Ring Assortment with O-Ring Gauge Bundle   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            75 Durometer. Manufactured from fluorocarbon rubber. Temperature range: -20° to 400°F. Has maximum resistance to deterioration by heat and fluids. Recognized by brown color.   
1                                                                                                                                                                                   

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [15]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [19]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 